# Regression Models: 
### Predicting GDP Annual Change.
---
Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA

In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

Read in Data

In [3]:
df = pd.read_csv('../../data/data_feat_engin.csv')

Look at columns for features.

In [4]:
df.columns

Index(['Year', 'Country', 'Refugees under UNHCR's mandate', 'Asylum-seekers',
       'IDPs of concern to UNHCR', 'Stateless persons', 'Others of concern',
       'Ref and Asyl', 'SUM REFUGEE', 'GDP_annual_change',
       'Adjusted savings: net national savings (current US$)',
       'Adjusted savings: particulate emission damage (current US$)',
       'Adolescent fertility rate (births per 1,000 women ages 15-19)',
       'Air transport, passengers carried',
       'Current health expenditure (% of GDP)',
       'Current health expenditure per capita (current US$)',
       'Death rate, crude (per 1,000 people)',
       'Domestic general government health expenditure per capita (current US$)',
       'Domestic private health expenditure per capita (current US$)',
       'Ease of doing business score (0 = lowest performance to 100 = best performance)',
       'Fixed broadband subscriptions (per 100 people)',
       'Fixed telephone subscriptions (per 100 people)',
       'GNI growth (ann

## Null Model
---

In [5]:
df['base'] = df['GDP_annual_change'].mean()
RMSE = np.sqrt(metrics.mean_squared_error(df['GDP_annual_change'], df['base']))
RMSE

2.4666729263695077

The Baseline model has a Root Mean Squared Error of 2.47%. On average the predicted GDP Annual Change is about \\2.47% off from the true sales price.

## Regression Pipelines
---
Will first look at all columns, not including feature engineering, and feed into a function for the various regression models. Interpretability will be important with this project, so it will be necessary to keep that in mind as a model is chosen to move forward.

In [6]:
features = ["Refugees under UNHCR's mandate", 
            'Asylum-seekers',
            'IDPs of concern to UNHCR', 
            'Stateless persons', 
            'Others of concern',
            'Ref and Asyl', 
            'SUM REFUGEE',
            'Adjusted savings: net national savings (current US$)',
            'Adjusted savings: particulate emission damage (current US$)',
            'Adolescent fertility rate (births per 1,000 women ages 15-19)',
            'Air transport, passengers carried',
            'Current health expenditure (% of GDP)',
            'Current health expenditure per capita (current US$)',
            'Death rate, crude (per 1,000 people)',
            'Domestic general government health expenditure per capita (current US$)',
            'Domestic private health expenditure per capita (current US$)',
            'Ease of doing business score (0 = lowest performance to 100 = best performance)',
            'Fixed broadband subscriptions (per 100 people)',
            'Fixed telephone subscriptions (per 100 people)',
            'GNI growth (annual %)',
            'International tourism, expenditures (current US$)',
            'International tourism, receipts (current US$)',
            'Military expenditure (current USD)', 'Population growth (annual %)',
            'Prevalence of undernourishment (% of population)',
            'Refugee population by country or territory of asylum',
            'Strength of legal rights index (0=weak to 12=strong)',
            'Unemployment, total (% of total labor force) (modeled ILO estimate)',
            'Net official flows from UN agencies: Total'
           ]

In [7]:
X = df[features]
y = df['GDP_annual_change']

In [8]:
def modelfunc(X, y):
    pipelines = [
        ('LINEAR REGRESSION', (Pipeline([ ('LR', LinearRegression())]))),
        ('DECISION TREE', (Pipeline([ ('TREE', DecisionTreeRegressor())]))),
        ('BAGGED TREE', (Pipeline([ ('BAG', BaggingRegressor())]))),
        ('RANDOM FOREST', (Pipeline([ ('RAND', RandomForestRegressor())]))),
        ('ADABOOST', (Pipeline([ ('ADA', AdaBoostRegressor())]))),
        ('KNN', (Pipeline([ ('sc', StandardScaler()),('KNN', KNeighborsRegressor())]))),
        ('LASSO',(Pipeline([ ('sc', StandardScaler()),('LASSO', LassoCV())]))),
        ('RIDGE',(Pipeline([ ('sc', StandardScaler()),('RIDGE', RidgeCV())])))
    ]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42) 
    for pipe_name ,model in pipelines:
        print(pipe_name)
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        trainscore = model.score(X_train, y_train)
        testscore = model.score(X_test, y_test)
        crossval = cross_val_score(model, X_train, y_train).mean()
        rmsetr= np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
        rmsete = np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))
        print (f'Model = {model}')
        print (f'Train Score = {trainscore}')
        print (f'Test Score = {testscore}')
        print (f'Cross Val Score = {crossval}')
        print (f'RMSE Train = {rmsetr}')
        print (f'RMSE Test = {rmsete}')
        print('')
        print('')

In [9]:
modelfunc(X, y)

LINEAR REGRESSION
Model = Pipeline(steps=[('LR', LinearRegression())])
Train Score = 0.5840815091198508
Test Score = 0.5543266541444651
Cross Val Score = 0.48571910010912883
RMSE Train = 1.628465335894157
RMSE Test = 1.5240839913050004


DECISION TREE
Model = Pipeline(steps=[('TREE', DecisionTreeRegressor())])
Train Score = 1.0
Test Score = 0.45149733872935605
Cross Val Score = 0.2909406060438142
RMSE Train = 5.781549060871421e-17
RMSE Test = 1.6907910499496495


BAGGED TREE
Model = Pipeline(steps=[('BAG', BaggingRegressor())])
Train Score = 0.9309785532871824
Test Score = 0.6745732158267024
Cross Val Score = 0.5146423401445237
RMSE Train = 0.6633862003221883
RMSE Test = 1.302348511557629


RANDOM FOREST
Model = Pipeline(steps=[('RAND', RandomForestRegressor())])
Train Score = 0.9452098065675193
Test Score = 0.740571813394214
Cross Val Score = 0.5719595255854927
RMSE Train = 0.5910521765645292
RMSE Test = 1.1628110515193768


ADABOOST
Model = Pipeline(steps=[('ADA', AdaBoostRegressor()

### Random Forest performed the best according to the function above. Dig into it with some tuning below.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42) 

rand = RandomForestRegressor()
# rand.fit(X_train, y_train)

params = {
    'n_estimators': [75, 100, 125],
    'max_depth': [None, 3, 4, 5], 
    'max_features': ['auto', 'sqrt', 3, 4, 5]
}
gs = GridSearchCV(RandomForestRegressor(), 
                 param_grid = params, 
                 verbose = 1)

gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)



Fitting 5 folds for each of 60 candidates, totalling 300 fits
0.582186778171328
{'max_depth': None, 'max_features': 5, 'n_estimators': 100}


## Below Changing features and running through the same function.
---
Adding engineered features.

In [7]:
features = ["Refugees under UNHCR's mandate", 
            'Asylum-seekers',
            'IDPs of concern to UNHCR', 
            'Stateless persons', 
            'Others of concern',
            'Ref and Asyl', 
            'SUM REFUGEE',
            'Adjusted savings: net national savings (current US$)',
            'Adjusted savings: particulate emission damage (current US$)',
            'Adolescent fertility rate (births per 1,000 women ages 15-19)',
            'Air transport, passengers carried',
            'Current health expenditure (% of GDP)',
            'Current health expenditure per capita (current US$)',
            'Death rate, crude (per 1,000 people)',
            'Domestic general government health expenditure per capita (current US$)',
            'Domestic private health expenditure per capita (current US$)',
            'Ease of doing business score (0 = lowest performance to 100 = best performance)',
            'Fixed broadband subscriptions (per 100 people)',
            'Fixed telephone subscriptions (per 100 people)',
            'GNI growth (annual %)',
            'International tourism, expenditures (current US$)',
            'International tourism, receipts (current US$)',
            'Military expenditure (current USD)', 'Population growth (annual %)',
            'Prevalence of undernourishment (% of population)',
            'Refugee population by country or territory of asylum',
            'Strength of legal rights index (0=weak to 12=strong)',
            'Unemployment, total (% of total labor force) (modeled ILO estimate)',
            'Net official flows from UN agencies: Total', 
            'Intl tourism expenditures recipts and asylum seekers', 
            'Intl tourism expenditures receipts and military',
            'net inflows UN and refugee pop', 
            'SUM REFUGEE and undernourishment'
           ]

In [8]:
X = df[features]
y = df['GDP_annual_change']

In [10]:
modelfunc(X, y)

LINEAR REGRESSION
Model = Pipeline(steps=[('LR', LinearRegression())])
Train Score = 0.002557638906477644
Test Score = -0.017215223763378917
Cross Val Score = -0.009983658394374938
RMSE Train = 2.3332791579773127
RMSE Test = 2.844216854015721


DECISION TREE
Model = Pipeline(steps=[('TREE', DecisionTreeRegressor())])
Train Score = 1.0
Test Score = 0.38441750489709114
Cross Val Score = 0.19975755776854878
RMSE Train = 8.176345093409794e-17
RMSE Test = 2.212582182463684


BAGGED TREE
Model = Pipeline(steps=[('BAG', BaggingRegressor())])
Train Score = 0.937558295750739
Test Score = 0.5176805071783713
Cross Val Score = 0.5607979968547073
RMSE Train = 0.583794731542818
RMSE Test = 1.9585005013680008


RANDOM FOREST
Model = Pipeline(steps=[('RAND', RandomForestRegressor())])
Train Score = 0.9509268605177158
Test Score = 0.5738923196024415
Cross Val Score = 0.605176729258368
RMSE Train = 0.51754095033276
RMSE Test = 1.8408396543279764


ADABOOST
Model = Pipeline(steps=[('ADA', AdaBoostRegress

Again, the Random Forest is performing the best. 

## Changing Features again to look at those with the highest correlation.
---

In [13]:
features = ['GNI growth (annual %)', 
            'Stateless persons',
            'Strength of legal rights index (0=weak to 12=strong)',
            'Adjusted savings: net national savings (current US$)',
            'Adjusted savings: particulate emission damage (current US$)',
            'Adolescent fertility rate (births per 1,000 women ages 15-19)',
            'Population growth (annual %)', 
            'Unemployment, total (% of total labor force) (modeled ILO estimate)',
            'Fixed telephone subscriptions (per 100 people)',
            'Current health expenditure (% of GDP)',
            'Current health expenditure per capita (current US$)',
            'Domestic general government health expenditure per capita (current US$)',
            'Fixed broadband subscriptions (per 100 people)',
            'Domestic private health expenditure per capita (current US$)',
            'International tourism, expenditures (current US$)',
            "Refugees under UNHCR's mandate", 
            'Asylum-seekers',
            'IDPs of concern to UNHCR', 
            'Others of concern',
            'Ref and Asyl', 
            'SUM REFUGEE',
            'Intl tourism expenditures recipts and asylum seekers', 
            'Intl tourism expenditures receipts and military',
            'net inflows UN and refugee pop', 
            'SUM REFUGEE and undernourishment'
           ]

In [14]:
X = df[features]
y = df['GDP_annual_change']

In [15]:
modelfunc(X, y)

LINEAR REGRESSION
Model = Pipeline(steps=[('LR', LinearRegression())])
Train Score = 0.002557638906477644
Test Score = -0.017215223763378695
Cross Val Score = -0.010641886204069362
RMSE Train = 2.3332791579773127
RMSE Test = 2.844216854015721


DECISION TREE
Model = Pipeline(steps=[('TREE', DecisionTreeRegressor())])
Train Score = 1.0
Test Score = 0.4303484964459777
Cross Val Score = 0.32386643883881705
RMSE Train = 8.176345093409794e-17
RMSE Test = 2.1284375098613215


BAGGED TREE
Model = Pipeline(steps=[('BAG', BaggingRegressor())])
Train Score = 0.9442624993137197
Test Score = 0.5322415590101071
Cross Val Score = 0.5590195461727404
RMSE Train = 0.55156480045986
RMSE Test = 1.9287107302222044


RANDOM FOREST
Model = Pipeline(steps=[('RAND', RandomForestRegressor())])
Train Score = 0.9555223690941599
Test Score = 0.5529658976379341
Cross Val Score = 0.5959314541454466
RMSE Train = 0.4927125457467094
RMSE Test = 1.8855003068713019


ADABOOST
Model = Pipeline(steps=[('ADA', AdaBoostRegr

Again, Random Forest is produces the best results.

Doing a GridSearch below to see if this can be increased at all from a cross val r2 score of .59.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42) 

rand = RandomForestRegressor()
# rand.fit(X_train, y_train)

params = {
    'n_estimators': [75, 100, 125, 150, 175, 200],
    'max_depth': [None, 3, 4, 5], 
    'max_features': ['auto', 'sqrt', 3, 4, 5]
}
gs = GridSearchCV(RandomForestRegressor(), 
                 param_grid = params, 
                 verbose = 1)

gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
0.6063038257293292
{'max_depth': None, 'max_features': 'auto', 'n_estimators': 100}


In [21]:
rand = RandomForestRegressor(n_estimators=100, max_depth=None, max_features='auto')
rand.fit(X_train, y_train)
y_pred_train = rand.predict(X_train)
y_pred_test = rand.predict(X_test)
trainscore = rand.score(X_train, y_train)
testscore = rand.score(X_test, y_test)
crossval = cross_val_score(rand, X_train, y_train).mean()
rmsetr= np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
rmsete = np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))

print (f'Train Score = {trainscore}')
print (f'Test Score = {testscore}')
print (f'Cross Val Score = {crossval}')
print (f'RMSE Train = {rmsetr}')
print (f'RMSE Test = {rmsete}')

Train Score = 0.9501705708913177
Test Score = 0.5706035408117434
Cross Val Score = 0.5878368358283506
RMSE Train = 0.5215137377991703
RMSE Test = 1.8479299719375515


In [18]:
feat_import = rf_feat_importance(gs, X)
feat_import[:10]

NameError: name 'rf_feat_importance' is not defined

## PCA
---

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    # stratify=y,
                                                    random_state = 42)

In [25]:
# Instantiate PCA with 20 components.
pca = PCA(n_components=20)

# Fit PCA to training data.
pca.fit(X_train)

PCA(n_components=20)

In [29]:
# Instantiate linear regression model.
lr = LinearRegression()

# Transform Z_train and Z_test.
Z_train = pca.transform(X_train)
Z_test = pca.transform(X_test)

# Fit on Z_train.

lr.fit(Z_train, y_train)

# Score on training and testing sets.
print(f'Training Score: {round(lr.score(Z_train, y_train),4)}')
print(f'Testing Score: {round(lr.score(Z_test, y_test),4)}')
print(f'Train Cross Val; : {round(cross_val_score(lr,Z_train, y_train).mean(),4  )}')
print(f'Test Cross Val; : {round(cross_val_score(lr,Z_test, y_test).mean(),4  )}')

Training Score: 0.1403
Testing Score: 0.089
Train Cross Val; : -0.0071
Test Cross Val; : -0.8063


## Stacking
---

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=42)

In [ ]:
level1_models = [
    ('GRAD', GradientBoostingClassifier()),
    ('ADA', AdaBoostClassifier()),
    ('LR', LinearRegression())
]


stack = StackingClassifier(estimators = level1_models, final_estimator = LinearRegression())


stack.fit(X_train, y_train)
trainscore = stack.score(X_train, y_train)
testscore = stack.score(X_test, y_test)
crossval = cross_val_score(stack, X_train, y_train).mean()
y_pred_train = stack.predict(X_train)
y_pred_test = stack.predict(X_test)
mse_train = metrics.mean_squared_error(y_train, y_pred_train)
mse_test = metrics.mean_absolute_error(y_test, y_pred_test)


print(f'Train: {trainscore}, Test: {testscore}, CV: {crossval}')
print (f'Mean Squared Error - Train = {mse_train}')
print (f'Mean Squared Error - Test = {mse_test}')